In [20]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
import itertools 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntSlider, Output
from IPython.display import clear_output
import h5py
import re
import matplotlib
import ipysheet

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.leap_tools import ccapt_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [22]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc)

Dropdown(description='Data mode:', options=('surface_concept', 'roentdec', 'leap_epos', 'leap_pos', 'ato_v6'),…

$$\textbf{You can specify which dataset to use in below block}$$

In [23]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'ions_selection_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')

In [25]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/OLO_AL_6_data
The dataset name after saving is: ions_selection_tof_calibration_cropped_OLO_AL_6_data
The figures will be saved on the path: D:/pyccapt/tests/data/OLO_AL_6_data//ions_selection/
Total number of Ions: 1865061


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,27.527876,27.400991,5263.160645,5263.160645,24501,615.666522,629.290080,-0.544408,-1.117469,0,0
1,0.0,0.0,0.0,27.048185,27.451994,5263.160645,5263.160645,25308,610.885941,642.745476,-0.977388,2.480082,807,2
2,0.0,0.0,0.0,27.001281,27.008272,5263.160645,5263.160645,25411,610.416149,625.195854,0.780000,0.977388,103,1
3,0.0,0.0,0.0,27.218993,27.270453,5263.160645,5263.160645,25674,612.593216,636.964182,0.971020,2.120327,263,1
4,0.0,0.0,0.0,27.592954,26.977407,5263.160645,5263.160645,26234,616.311750,642.718044,2.072571,-2.247673,560,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865056,0.0,0.0,0.0,27.130503,26.662453,5710.793945,5710.793945,24534,611.709426,613.146348,1.579102,-2.365469,2,1
1865057,0.0,0.0,0.0,27.280111,27.800477,5710.793945,5710.793945,24644,613.202760,620.381538,-2.091673,1.286204,110,1
1865058,0.0,0.0,0.0,27.031051,27.405944,5710.793945,5710.793945,24774,610.714381,609.010974,-1.505878,0.684490,130,1
1865059,0.0,0.0,0.0,26.971629,26.813825,5710.793945,5710.793945,25014,610.118935,623.460780,2.747510,2.142612,240,1


In [42]:
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [43]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(True));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_7828\4163154723.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [74]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=100, description='peak prominance:')
distance=widgets.IntText(value=500, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot(variables, plot):
    
    with out:
        clear_output(True)
    
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.mc_hist_plot(variables, bin_size_p, mode='mc_c', prominence=prominence_p, distance=distance_p, percent=percent_p, selector='range', plot=True, figname=index_fig_p, lim=lim_tof_p,
                          peaks_find_plot=plot_peak_p)

In [75]:
peak_val = widgets.FloatText(value=1.1, description='peak value:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=1, description='aboundance threshold:')
mode = widgets.Dropdown(
    options=[('elements', 'elements'), ('molecules', 'molecules')],
    value='elements',
    description='mode:'
)

num_element = widgets.IntText(value=1, description='num element:')
# element_threshold = widgets.FloatText(value=1.0, description='Aboundance threshold:')

def element_finder():
    with out:
        peak_val_s = peak_val.value
        charge_s = charge.value
        num_element_s = num_element.value
#         num_molecule_s = num_molecule.value
        mode_s = mode.value
#         element_threshold_s = element_threshold.value
        aboundance_threshold_s = aboundance_threshold.value
        clear_output(True)
        if mode_s == 'elements':
            df = ion_selection.find_close_element(peak_val_s, num_element_s, aboundance_threshold_s, charge=charge_s)
        elif mode_s == 'molecules':
            df = ion_selection.find_close_molecule(peak_val_s, num_element_s, aboundance_threshold_s, charge=charge_s)
        display(df)

In [76]:
isotope_formula = widgets.Text(
    value='',
    placeholder='Type a formula',
    description='Isotope formula:',
    disabled=False
)

def manual_formula_calculator():
    isotopeTableFile = '../../../files/isotopeTable.h5'
    dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
    df = ion_selection.molecule_isotop_list(dataframe, isotope_formula.value)
    with out:
        df = ion_selection.molecule_isotop_list(dataframe, isotope_formula.value)
        clear_output(True)
        display(df)

In [77]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']
dropdownDict = {'element': elementsList, 'isotope': elementIsotopeList}

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]), "{}({})".format(element[0], element[1]))
    dropdownList.append(tupleElement)

dropdown = wd.dropdownWidget(dropdownList, "Elements")
dropdown.observe(wd.on_change_ions_selection)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
chargeDropdown = wd.dropdownWidget(chargeList, "Charge")
chargeDropdown.observe(wd.on_change_charge_ions_selection)

# wd.compute_element_isotope_values_according_to_selected_charge(mode='ions_selection')

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

def buttonAdd_f(b,):
    with out:
        clear_output(True)
        wd.onClickAdd(b,)
        display()
def buttonDelete_f(b,):
    with out:
        clear_output(True)
        variables.listMaterial.pop()
        variables.element.pop()
        variables.isotope.pop()
        variables.charge.pop()
        print("Updated List : ", variables.listMaterial)
        print("Updated element List : ", variables.element)
        print("Updated isotope List : ",  variables.isotope)
        print("Updated charge List : ", variables.charge)
def buttonResett_f(b,):
    with out:
        clear_output(True)
        wd.onClickReset(b,)
        display()

buttonAdd.on_click(buttonAdd_f)
buttonDelete.on_click(buttonDelete_f)
buttonReset.on_click(buttonResett_f)

add_button = widgets.Button(
    description='add molecule',
)

peak_molecule = widgets.Text(
    value='',
    placeholder='molecule: H(1)2O(16)(2+)',
    description='molecule:',
    disabled=False
)
peak_molecule_elements = widgets.Text(
    value='',
    placeholder='elements: H,O',
    description='elements:',
    disabled=False
)
peak_molecule_isotope = widgets.Text(
    value='',
    placeholder='isotope: 1,16',
    description='isotope:',
    disabled=False
)
peak_molecule_charge = widgets.IntText(
    placeholder='charge: 3',
    description='charge:',
    disabled=False
)

@add_button.on_click
def add_molucule_to(b,):
    with out:
        clear_output(True)
        if peak_molecule.value == 'unranged':
            variables.listMaterial.append(peak_molecule.value)
            variables.element.append(peak_molecule.value)
            variables.isotope.append(0)
            variables.charge.append(0)
        else:
            variables.listMaterial.append(peak_molecule.value)
            variables.element.append(peak_molecule_elements.value.split(','))
            variables.isotope.append(peak_molecule_isotope.value.split(','))
            variables.charge.append(peak_molecule_charge.value)
        print("Updated List : ", variables.listMaterial)
        print("Updated element List : ", variables.element)
        print("Updated isotope List : ",  variables.isotope)
        print("Updated charge List : ", variables.charge)

In [91]:
plot_button = widgets.Button(
    description='plot hist',
)

find_button = widgets.Button(
    description='find element',
)

formula_find_button = widgets.Button(
    description='molecule calculate',
)

plot_fdm_button = widgets.Button(
    description='plot FDM',
)
bin_fdm = widgets.IntText(value=256, description='bin FDM:')

@plot_fdm_button.on_click
def plot_fdm_on_click(b, variables=variables, save=True):
    with out:
        clear_output(True)
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected tof are: (%s, %s)' %(variables.selected_x1, variables.selected_x2)) 
            mask_temporal = np.logical_and((variables.dld_t_calib > variables.selected_x1),
                                           (variables.dld_t_calib < variables.selected_x2))
            x = variables.dld_x_det[mask_temporal]
            y = variables.dld_y_det[mask_temporal]
            bin_fdm_p = bin_fdm.value
            index_fig_p = index_fig.value
            bins = [bin_fdm_p, bin_fdm_p]

            calibration.plot_fdm(x, y, variables, save, bins, index_fig_p)
            
@plot_button.on_click
def plot_on_click(b, figname='fff', plot=True):
    hist_plot(variables, plot)

    
@find_button.on_click
def vol_on_click(b,):
    element_finder()
    
@formula_find_button.on_click
def manual_formula(b,):
    manual_formula_calculator()
    
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, bin_fdm, plot_peak])
tab2 = HBox(children=[VBox(children=[peak_val, num_element, charge, aboundance_threshold, mode, 
                      find_button]),HBox(children=[isotope_formula, formula_find_button])])
tab3 = VBox(children=[dropdown, chargeDropdown, buttonAdd, buttonDelete, buttonReset])
tab4 = VBox(children=[peak_molecule, peak_molecule_elements, peak_molecule_isotope, peak_molecule_charge, add_button])


tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'mc plot')
tab.set_title(1, 'element finder')
tab.set_title(2, 'add ions')
tab.set_title(3, 'add molecules')


display(VBox(children=[tab,HBox(children=[plot_button, plot_fdm_button])]))
out = Output()
display(out)
# unranged	

Output()

In [79]:
fake = Factory.create()
data_table = '../../../files/color_scheme.h5'
dataframe = data_tools.read_hdf5_through_pandas(data_table)
color_list = []
for i in range(len(variables.listMaterial)):
    element_selec = re.sub("[\(\[].*?[\)\]]", "", variables.listMaterial[i])
    if len(re.findall(r'[A-Z]', element_selec)) > 1:
        try:
            r = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['r'].to_numpy()
            g = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['g'].to_numpy()
            b = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['b'].to_numpy()
            cc = matplotlib.colors.to_hex([r[0], g[0], b[0]])
            color_list.append(cc)
        except:
            print('The element is not clor list')
            color_list.append(fake.hex_color())
    else:
        try:
            r = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['r'].to_numpy()
            g = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['g'].to_numpy()
            b = dataframe[dataframe['ion'] == re.sub(r'[0-9]', '', element_selec)]['b'].to_numpy()
            cc = matplotlib.colors.to_hex([r[0], g[0], b[0]])
            color_list.append(cc)
        except:
            print('The element is not clor list')
            color_list.append(fake.hex_color())

In [33]:
peak_width_chos = []
for i in range(len(variables.peaks_idx)):
    peak_width_chos.append(variables.peak_width[variables.peaks_idx[i]])
peak_width_chos = np.array(peak_width_chos)

In [34]:
range_data = pd.DataFrame({"element": variables.element, "isotope": variables.isotope, "charge": variables.charge,
        "mc_low": [item[1] for item in peak_width_chos],
       "mc_up": [item[2] for item in peak_width_chos],"color": color_list})

In [35]:
sheet7 = ipysheet.from_dataframe(range_data)
sheet7

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=2, row_start=0, squeeze_row=False…

In [38]:
range_data = ipysheet.to_dataframe(sheet7)
range_data

,element,isotope,charge,mc_low,mc_up,color
0,H,1,1,0.802639,1.103087,#b2aa2d
1,Al,27,2,13.000000,13.950000,#e7e0d1
2,Al,27,1,26.000000,29.000000,#e7e0d1


In [40]:
def mc_hist_plot(data, range_data, bin_size, prominence, distance, percent, selector, plot, figname, lim_mc, mc_peak_label, ranging, adjust_label):
    mc = data['mc_c (Da)'].to_numpy()
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(mc[mc < lim_mc], bin_size, range_data=range_data, mc_peak_label=mc_peak_label, ranging=ranging, adjust_label=adjust_label, distance=distance, percent=percent, prominence=prominence, selector=selector, plot=plot, label='mc', fig_name=figname+'_ini')
    index_max_ini = np.argmax(peaks_y_ini)
    variables.max_peak = peaks_ini[index_max_ini]
    variables.peak = peaks_ini
    mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_ini)):
        print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

interact_manual(mc_hist_plot, data=fixed(data), range_data=fixed(range_data), bin_size=widgets.FloatText(value=0.1)
               ,prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), lim_mc=widgets.IntText(value=150)
               ,percent=widgets.IntText(value=50), selector=fixed('rect'), plot=fixed(True), figname=fixed(figname),
               mc_peak_label=widgets.Dropdown(options=[('True', True), ('False', False)]),
                ranging=widgets.Dropdown(options=[('True', True), ('False', False)]),
                adjust_label=widgets.Dropdown(options=[('True', True), ('False', False)]));


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=50, description='prominence'…

In [42]:
range_data

,element,isotope,charge,mc_low,mc_up,color
0,H,1,1,0.802639,1.103087,#b2aa2d
1,Al,27,2,13.000000,13.950000,#e7e0d1
2,Al,27,1,26.000000,29.000000,#e7e0d1


In [43]:
range_data.dtypes

element     object
isotope      int32
charge       int32
mc_low     float64
mc_up      float64
color       object
dtype: object

In [44]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, range_data, 'df')

In [45]:
# save data in csv format
data_tools.store_df_to_csv(range_data, variables.result_path + 'range_' + dataset_name + '.csv')